In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import numpy as np

# Path to dataset
train_path = "/kaggle/input/amazon-2025-ml-dataset/cleaned_data.csv"

# Load the data
df = pd.read_csv(train_path)

# Display basic info
print("Shape of dataset:", df.shape)
df.head()

Shape of dataset: (75000, 9)


,sample_id,catalog_content,image_link,price,item_name,bullet_points,product_description,value,unit
0,33127,"Item Name: La Victoria Green Taco Sauce Mild, ...",https://m.media-amazon.com/images/I/51mo8htwTH...,4.89,"La Victoria Green Taco Sauce Mild, 12 Ounce (P...",NaN,NaN,72.00,Fl Oz
1,198967,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.12,"Salerno Cookies, The Original Butter Cookies, ...",Original Butter Cookies: Classic butter cookie...,NaN,32.00,Ounce
2,261251,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97,"Bear Creek Hearty Soup Bowl, Creamy Chicken wi...",Loaded with hearty long grain wild rice and ve...,NaN,11.40,Ounce
3,55858,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34,Judee’s Blue Cheese Powder 11.25 oz - Gluten-F...,"Add to your favorite appetizers, dips & spread...",Judees Powdered Blue Cheese cheddar cheese pow...,11.25,Ounce
4,292686,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",https://m.media-amazon.com/images/I/41sA037+Qv...,66.49,"kedem Sherry Cooking Wine, 12.7 Ounce - 12 per...",NaN,NaN,12.00,Count


In [2]:
# Check unique unit types and their frequencies
unit_counts = (
    df['unit']
    .astype(str)                   # convert NaN to 'nan' string for counting
    .str.strip()
    .value_counts(dropna=False)
)

print("🔹 Total unique units:", len(unit_counts))
print("\n🔹 Top 87 most frequent units:\n")
print(unit_counts.head(87))


🔹 Total unique units: 87

🔹 Top 87 most frequent units:

unit
Ounce       40982
Count       17452
Fl Oz       11082
ounce        1960
oz            962
            ...  
ltr             1
PACK            1
can             1
Carton          1
Tea bags        1
Name: count, Length: 87, dtype: int64


In [3]:
# Count all unique unit types
unit_counts = (
    df['unit']
    .astype(str)
    .str.strip()
    .value_counts(dropna=False)
)

print(f"🔹 Total unique units: {len(unit_counts)}\n")
print("🔹 All unique units with counts:\n")

# Print all without truncation
for unit, count in unit_counts.items():
    print(f"{unit:25s} : {count}")


🔹 Total unique units: 87

🔹 All unique units with counts:

Ounce                     : 40982
Count                     : 17452
Fl Oz                     : 11082
ounce                     : 1960
oz                        : 962
nan                       : 952
count                     : 741
pound                     : 158
fl oz                     : 128
Oz                        : 78
ct                        : 46
Pound                     : 44
Fluid Ounce               : 30
lb                        : 30
FL Oz                     : 28
OZ                        : 24
Fl. Oz                    : 21
gram                      : 20
Gram                      : 19
COUNT                     : 16
FL OZ                     : 15
each                      : 14
ounces                    : 14
Each                      : 12
packs                     : 11
millilitre                : 9
fluid ounce               : 8
fluid ounces              : 8
Liters                    : 7
Pack                      : 6


In [4]:
unit_map = {
    # weight
    'ounce':'ounce', 'ounces':'ounce', 'oz':'ounce', 'oz.':'ounce', 'fl oz':'fl_ounce',
    'Ounce':'ounce', 'OZ':'ounce', 'Fl Oz':'fl_ounce', 'FL Oz':'fl_ounce', 'Fl. Oz':'fl_ounce',
    'Fluid Ounce':'fl_ounce', 'Fluid Ounces':'fl_ounce', 'fluid ounce':'fl_ounce',
    'fluid ounces':'fl_ounce', 'Fl oz':'fl_ounce', 'fl. oz.':'fl_ounce',
    'pound':'pound', 'Pound':'pound', 'pounds':'pound', 'Pounds':'pound',
    'lb':'pound', 'LB':'pound',
    'gram':'gram', 'grams':'gram', 'Gram':'gram', 'Grams':'gram', 'gramm':'gram',
    'kg':'kilogram', 'K':'kilogram',

    # volume
    'fl oz':'fl_ounce', 'Fl Oz':'fl_ounce', 'FL OZ':'fl_ounce', 'Fl.oz':'fl_ounce',
    'milliliter':'milliliter', 'millilitre':'milliliter', 'mililitro':'milliliter', 'ml':'milliliter', 'ltr':'liter', 'Liters':'liter',

    # count / pieces
    'count':'count', 'Count':'count', 'COUNT':'count', 'ct':'count', 'CT':'count',
    'unit':'count', 'units':'count', 'Piece':'count', 'Each':'count', 'each':'count',
    'pack':'count', 'Pack':'count', 'PACK':'count', 'packs':'count', 'Packs':'count',
    'Per Package':'count', 'per Box':'count', 'per Carton':'count', 'Comes as a single':'count',

    # containers / packaging
    'bottle':'container', 'Bottle':'container', 'bottles':'container',
    'Bag':'container', 'bag':'container', 'Box':'container', 'box':'container',
    'Carton':'container', 'Jar':'container', 'Pouch':'container',
    'Bucket':'container', 'Can':'container', 'case':'container', 'CASE':'container',

    # others
    'cm':'misc', 'in':'misc', 'Sq Ft':'misc', 'sq ft':'misc', 'Foot':'misc',
    'product':'misc', 'Tea Bags':'misc', 'Tea bags':'misc', 'Paper Cupcake Liners':'misc',
    'Ziplock bags':'misc', 'Bullet Point':'misc'
}


In [5]:
df['unit_clean'] = df['unit'].map(unit_map).fillna('misc')


In [6]:
print(df['unit_clean'].value_counts())


unit_clean
ounce         43942
count         18317
fl_ounce      11332
misc           1057
pound           240
gram             49
container        31
milliliter       15
kilogram          9
liter             8
Name: count, dtype: int64


In [7]:
def map_unit_class(u):
    if u in ['ounce', 'fl_ounce', 'gram', 'milliliter', 'pound', 'kilogram', 'liter']:
        return 'quantity'
    elif u in ['count', 'container']:
        return 'count'
    else:
        return 'misc'

df['unit_class'] = df['unit_clean'].apply(map_unit_class)

In [8]:
unit_to_gram_equiv = {
    'ounce': 28.35,
    'fl_ounce': 29.57,    # ml equivalent
    'pound': 453.59,
    'gram': 1.0,
    'milliliter': 1.0,
    'kilogram': 1000.0,
    'liter': 1000.0
}

df['std_value'] = df.apply(
    lambda x: x['value'] * unit_to_gram_equiv.get(x['unit_clean'], np.nan)
    if x['unit_class'] == 'quantity' else x['value'],
    axis=1
)

In [9]:
df['log_value'] = np.log1p(df['value'])
df['log_std_value'] = np.log1p(df['std_value'])
df['price_per_value'] = df['price'] / df['value'].replace(0, np.nan)
df['price_per_std_value'] = df['price'] / df['std_value'].replace(0, np.nan)

/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [10]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df[['price_per_value', 'price_per_std_value']] = df[['price_per_value', 'price_per_std_value']].fillna(df[['price_per_value', 'price_per_std_value']].median())


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Combine text columns (simple but effective)
df['text_all'] = (
    df['item_name'].fillna('') + ' ' +
    df['bullet_points'].fillna('') + ' ' +
    df['product_description'].fillna('')
)

tfidf = TfidfVectorizer(
    max_features=50000,     # tune for speed vs performance
    ngram_range=(1, 2),     # capture single words and pairs
    stop_words='english',
    min_df=3
)

X_text = tfidf.fit_transform(df['text_all'])
print("TF-IDF matrix shape:", X_text.shape)


TF-IDF matrix shape: (75000, 50000)


In [12]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from scipy.sparse import hstack, csr_matrix

# Numeric columns (fill missing and scale)
num_cols = ['value', 'log_value', 'std_value', 'log_std_value']
scaler = StandardScaler()
X_num = scaler.fit_transform(df[num_cols].fillna(0))

# Categorical (one-hot encode units)
cat_cols = ['unit_clean', 'unit_class']
ohe = OneHotEncoder(handle_unknown='ignore', sparse=True)
X_cat = ohe.fit_transform(df[cat_cols])

# Combine all
X_final = hstack([X_text, csr_matrix(X_num), X_cat])
print("Final feature matrix:", X_final.shape)


Final feature matrix: (75000, 50017)


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [13]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import numpy as np

X_train, X_val, y_train, y_val = train_test_split(
    X_final, df['price'], test_size=0.2, random_state=42
)

model = Ridge(alpha=1.0, solver="lsqr", random_state=42)
model.fit(X_train, y_train)
preds = model.predict(X_val)

# Compute SMAPE
smape = np.mean(np.abs(preds - y_val) / ((np.abs(preds) + np.abs(y_val)) / 2)) * 100
print(f"Validation SMAPE: {smape:.2f}%")


Validation SMAPE: 70.92%


In [14]:
from sklearn.model_selection import train_test_split
import numpy as np

# Split data
X_train, X_val, y_train, y_val = train_test_split(
    X_final, df['price'], test_size=0.2, random_state=42
)

# 🔹 Log-transform target
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

# Train Ridge on log(price)
model = Ridge(alpha=1.0, solver="lsqr", random_state=42)
model.fit(X_train, y_train_log)

# Predict in log-space
preds_log = model.predict(X_val)

# Convert back to price scale
preds = np.expm1(preds_log)

# Compute SMAPE on actual prices
smape = np.mean(
    np.abs(preds - y_val) / ((np.abs(preds) + np.abs(y_val)) / 2)
) * 100

print(f"Validation SMAPE (log target): {smape:.2f}%")

Validation SMAPE (log target): 54.17%


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

def smape_score(y_true, y_pred):
    return np.mean(np.abs(y_pred - y_true) / ((np.abs(y_pred) + np.abs(y_true)) / 2)) * 100

def train_and_evaluate(model, X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # log target
    y_train_log = np.log1p(y_train)
    model.fit(X_train, y_train_log)
    preds = np.expm1(model.predict(X_val))

    smape = smape_score(y_val, preds)
    print(f"{model.__class__.__name__} SMAPE: {smape:.2f}%")
    return smape


In [18]:
import lightgbm as lgb

lgb_model = lgb.LGBMRegressor(
    objective='regression',
    learning_rate=0.05,
    n_estimators=2000,
    num_leaves=64,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

train_and_evaluate(lgb_model, X_final, df['price'])


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 17.159743 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1165526
[LightGBM] [Info] Number of data points in the train set: 60000, number of used features: 36273
[LightGBM] [Info] Start training from score 2.740904
LGBMRegressor SMAPE: 51.05%


51.05451688857253

In [17]:
from sklearn.linear_model import ElasticNet

elastic = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42)
train_and_evaluate(elastic, X_final, df['price'])


ElasticNet SMAPE: 72.13%


72.13108879404909

In [15]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=2000,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    tree_method='hist',  # use 'gpu_hist' if GPU available
    random_state=42
)

train_and_evaluate(xgb_model, X_final, df['price'])


KeyboardInterrupt: 

In [ ]:
from catboost import CatBoostRegressor

# ✅ GPU version of CatBoost
cat_model = CatBoostRegressor(
    iterations=2000,
    learning_rate=0.05,
    depth=8,
    loss_function='RMSE',
    task_type='GPU',          # 🔹 Use GPU instead of CPU
    devices='0',              # 🔹 Use first GPU (change to '0:1' for multi-GPU)
    random_seed=42,
    verbose=200
)

train_and_evaluate(cat_model, X_final, df['price'])



In [ ]:
from sklearn.linear_model import Ridge

ridge_model = Ridge(alpha=1.0, solver="lsqr", random_state=42)

# ✅ Run training + evaluation
ridge_smape = train_and_evaluate(ridge_model, X_final, df["price"])

In [1]:
!pip install -U "transformers==4.44.2" "sentence-transformers==3.0.1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 83.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 67.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 M

In [4]:
# ================================================================
# 1️⃣ Imports
# ================================================================
import pandas as pd
import numpy as np
import re, torch
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from scipy.sparse import hstack, csr_matrix
from tqdm.auto import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)

# ================================================================
# 2️⃣ Load Dataset
# ================================================================
train_path = "/kaggle/input/amazon-2025-ml-dataset/train.csv"
df = pd.read_csv(train_path)
print(f"✅ Loaded dataset: {df.shape[0]:,} rows × {df.shape[1]} columns")

# ================================================================
# 3️⃣ Extract Structured Fields from catalog_content
# ================================================================
def extract_features(text):
    if pd.isna(text):
        return pd.Series({
            "item_name": None,
            "bullet_points": None,
            "product_description": None,
            "value": None,
            "unit": None
        })
    item_names = re.findall(r"Item Name(?:\s*\d*)?:\s*(.*?)\s(?=Item Name|Bullet Point|Product Description:|Value:|Unit:|$)", text, re.S)
    item_name = item_names[-1].strip() if item_names else None

    bullet_points = re.findall(r"Bullet Point \d*:\s*(.*?)\s(?=Bullet Point \d+|Product Description:|Value:|Unit:|$)", text, re.S)
    bullet_points = " | ".join(bp.strip() for bp in bullet_points if bp.strip()) or None

    desc_match = re.search(r"Product Description:\s*(.*?)\s(?=Value:|Unit:|$)", text, re.S)
    product_description = desc_match.group(1).strip() if desc_match else None

    value_match = re.search(r"Value:\s*([0-9]*\.?[0-9]+)", text)
    value = float(value_match.group(1)) if value_match else None

    unit_match = re.search(r"Unit:\s*([A-Za-z\s\.]+)", text)
    unit = unit_match.group(1).strip() if unit_match else None

    return pd.Series({
        "item_name": item_name,
        "bullet_points": bullet_points,
        "product_description": product_description,
        "value": value,
        "unit": unit
    })

print("⏳ Extracting structured features...")
features = df["catalog_content"].apply(extract_features)
df = pd.concat([df, features], axis=1)
print("✅ Feature extraction complete!")

# ================================================================
# 4️⃣ Unit Normalization
# ================================================================
unit_map = {
    # weight
    'ounce':'ounce', 'ounces':'ounce', 'oz':'ounce', 'oz.':'ounce', 'Ounce':'ounce',
    'OZ':'ounce', 'pound':'pound', 'Pound':'pound', 'pounds':'pound', 'Pounds':'pound',
    'lb':'pound', 'LB':'pound',
    'gram':'gram', 'grams':'gram', 'Gram':'gram', 'Grams':'gram', 'gramm':'gram',
    'kg':'kilogram', 'K':'kilogram',
    # volume
    'fl oz':'fl_ounce', 'Fl Oz':'fl_ounce', 'FL Oz':'fl_ounce', 'Fl.oz':'fl_ounce',
    'Fluid Ounce':'fl_ounce', 'Fluid Ounces':'fl_ounce', 'fluid ounce':'fl_ounce',
    'fluid ounces':'fl_ounce', 'milliliter':'milliliter', 'millilitre':'milliliter',
    'mililitro':'milliliter', 'ml':'milliliter', 'ltr':'liter', 'Liters':'liter',
    # count / pieces
    'count':'count', 'Count':'count', 'COUNT':'count', 'ct':'count', 'CT':'count',
    'unit':'count', 'units':'count', 'Piece':'count', 'Each':'count', 'each':'count',
    'pack':'count', 'Pack':'count', 'PACK':'count', 'packs':'count', 'Packs':'count',
    # containers
    'bottle':'container', 'Bottle':'container', 'bottles':'container',
    'Bag':'container', 'bag':'container', 'Box':'container', 'box':'container',
    'Carton':'container', 'Jar':'container', 'Pouch':'container',
    'Bucket':'container', 'Can':'container', 'case':'container', 'CASE':'container'
}
df['unit_clean'] = df['unit'].map(unit_map).fillna('misc')

def map_unit_class(u):
    if u in ['ounce', 'fl_ounce', 'gram', 'milliliter', 'pound', 'kilogram', 'liter']:
        return 'quantity'
    elif u in ['count', 'container']:
        return 'count'
    else:
        return 'misc'

df['unit_class'] = df['unit_clean'].apply(map_unit_class)

# ================================================================
# 5️⃣ Value Normalization Columns
# ================================================================
unit_to_gram_equiv = {
    'ounce': 28.35, 'fl_ounce': 29.57, 'pound': 453.59,
    'gram': 1.0, 'milliliter': 1.0, 'kilogram': 1000.0, 'liter': 1000.0
}

df['std_value'] = df.apply(
    lambda x: x['value'] * unit_to_gram_equiv.get(x['unit_clean'], np.nan)
    if x['unit_class'] == 'quantity' else x['value'],
    axis=1
)
df['log_value'] = np.log1p(df['value'])
df['log_std_value'] = np.log1p(df['std_value'])

# ================================================================
# 6️⃣ Combine Text Columns for DistilBERT
# ================================================================
df['text_all'] = (
    df['item_name'].fillna('') + ' ' +
    df['bullet_points'].fillna('') + ' ' +
    df['product_description'].fillna('')
)

# ================================================================
# 7️⃣ DistilBERT Tokenizer & Embeddings
# ================================================================
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
distilbert_model = DistilBertModel.from_pretrained("distilbert-base-uncased").to(device)
distilbert_model.eval()
print("✅ Loaded DistilBERT successfully!")

def get_distilbert_embedding(text, max_len=64):
    tokens = tokenizer(text, return_tensors="pt",
                       truncation=True, padding="max_length",
                       max_length=max_len)
    tokens = {k: v.to(device) for k, v in tokens.items()}
    with torch.no_grad():
        outputs = distilbert_model(**tokens)
    emb = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
    return emb

texts = df['text_all'].tolist()
distilbert_embeddings = np.vstack([get_distilbert_embedding(t) for t in tqdm(texts, desc="Encoding with DistilBERT")])
print("✅ DistilBERT embeddings:", distilbert_embeddings.shape)

# ================================================================
# 8️⃣ Numeric & Categorical Feature Matrices
# ================================================================
num_cols = ['value', 'log_value', 'std_value', 'log_std_value']
scaler = StandardScaler()
X_num = scaler.fit_transform(df[num_cols].fillna(0))

cat_cols = ['unit_clean', 'unit_class']
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_cat = ohe.fit_transform(df[cat_cols])

# Combine all
X_final = np.hstack([distilbert_embeddings, X_num, X_cat])
print("✅ Final feature matrix:", X_final.shape)

# ================================================================
# 9️⃣ Predict Using Your Trained Model (e.g., LightGBM)
# ================================================================
# Example placeholder — ensure you have a trained model object lgb_model
# preds_log = lgb_model.predict(X_final)
# preds = np.expm1(preds_log)
# preds = np.clip(preds, a_min=0.1, a_max=None)

# submission = pd.DataFrame({
#     "sample_id": df["sample_id"],
#     "price": preds
# })
# submission.to_csv("/kaggle/working/test_out.csv", index=False)
# print("✅ Saved submission → /kaggle/working/test_out.csv")


✅ Using device: cuda
✅ Loaded dataset: 75,000 rows × 4 columns
⏳ Extracting structured features...
✅ Feature extraction complete!


/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


✅ Loaded DistilBERT successfully!


Encoding with DistilBERT:   0%|          | 0/75000 [00:00<?, ?it/s]

✅ DistilBERT embeddings: (75000, 768)
✅ Final feature matrix: (75000, 785)


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [7]:
# ================================================================
# 9️⃣ Prepare Target and Train/Validation Split
# ================================================================
from sklearn.model_selection import train_test_split

y = df["price"].values
y_log = np.log1p(y)  # use log-transformed target

X_train, X_val, y_train, y_val = train_test_split(
    X_final, y_log, test_size=0.2, random_state=42
)

print("✅ Data split complete →")
print(f"Train: {X_train.shape}, Validation: {X_val.shape}")

# ================================================================
# 🔟 Convert to PyTorch Tensors and Dataloaders
# ================================================================
from torch.utils.data import DataLoader, TensorDataset

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).to(device)

train_loader = DataLoader(
    TensorDataset(X_train_tensor, y_train_tensor),
    batch_size=64, shuffle=True
)
val_loader = DataLoader(
    TensorDataset(X_val_tensor, y_val_tensor),
    batch_size=64, shuffle=False
)

# ================================================================
# 1️⃣1️⃣ Define Deep Regression Model
# ================================================================
import torch.nn as nn

class PriceRegressor(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.model(x).squeeze(-1)

model = PriceRegressor(X_final.shape[1]).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)
criterion = nn.L1Loss()  # MAE loss works well for SMAPE tasks
print(f"✅ Model initialized: {sum(p.numel() for p in model.parameters()):,} parameters")

# ================================================================
# 1️⃣2️⃣ SMAPE Metric Function
# ================================================================
def smape(y_true, y_pred):
    y_true, y_pred = np.expm1(y_true), np.expm1(y_pred)
    return np.mean(np.abs(y_pred - y_true) / ((np.abs(y_pred) + np.abs(y_true)) / 2)) * 100

# ================================================================
# 🧠 Training Loop with Early Stopping
# ================================================================
EPOCHS = 100
patience = 5
best_smape = float("inf")
epochs_no_improve = 0
best_model_state = None

for epoch in range(EPOCHS):
    # -----------------------------
    # 🔹 TRAIN PHASE
    # -----------------------------
    model.train()
    train_losses = []
    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

    # -----------------------------
    # 🔹 VALIDATION PHASE
    # -----------------------------
    model.eval()
    val_preds, val_true = [], []
    with torch.no_grad():
        for xb, yb in val_loader:
            preds = model(xb)
            val_preds.extend(preds.cpu().numpy())
            val_true.extend(yb.cpu().numpy())

    val_preds, val_true = np.array(val_preds), np.array(val_true)
    smape_score = smape(val_true, val_preds)
    avg_train_loss = np.mean(train_losses)

    print(f"Epoch {epoch+1:03d}/{EPOCHS} | Train Loss: {avg_train_loss:.4f} | Validation SMAPE: {smape_score:.2f}%")

    # -----------------------------
    # 🔹 EARLY STOPPING CHECK
    # -----------------------------
    if smape_score < best_smape - 0.1:  # small tolerance
        best_smape = smape_score
        epochs_no_improve = 0
        best_model_state = model.state_dict()
        torch.save(model.state_dict(), "/kaggle/working/best_distilbert_price_regressor.pt")
        print(f"💾 Saved new best model (SMAPE: {best_smape:.2f}%)")
    else:
        epochs_no_improve += 1
        print(f"⏳ No improvement for {epochs_no_improve} epoch(s)")

    if epochs_no_improve >= patience:
        print(f"🛑 Early stopping triggered after {epoch+1} epochs. Best SMAPE: {best_smape:.2f}%")
        break

# ================================================================
# ✅ Save Final Model (Best)
# ================================================================
if best_model_state:
    model.load_state_dict(best_model_state)
    torch.save(model.state_dict(), "/kaggle/working/best_distilbert_price_regressor.pt")

print("\n✅ Training complete — best model saved to /kaggle/working/best_distilbert_price_regressor.pt")



✅ Data split complete →
Train: (60000, 785), Validation: (15000, 785)
✅ Model initialized: 550,273 parameters
Epoch 001/100 | Train Loss: 0.7473 | Validation SMAPE: 64.25%
💾 Saved new best model (SMAPE: 64.25%)
Epoch 002/100 | Train Loss: 0.6574 | Validation SMAPE: 62.26%
💾 Saved new best model (SMAPE: 62.26%)
Epoch 003/100 | Train Loss: 0.6391 | Validation SMAPE: 61.39%
💾 Saved new best model (SMAPE: 61.39%)
Epoch 004/100 | Train Loss: 0.6286 | Validation SMAPE: 61.00%
💾 Saved new best model (SMAPE: 61.00%)
Epoch 005/100 | Train Loss: 0.6214 | Validation SMAPE: 61.37%
⏳ No improvement for 1 epoch(s)
Epoch 006/100 | Train Loss: 0.6132 | Validation SMAPE: 59.85%
💾 Saved new best model (SMAPE: 59.85%)
Epoch 007/100 | Train Loss: 0.6082 | Validation SMAPE: 58.98%
💾 Saved new best model (SMAPE: 58.98%)
Epoch 008/100 | Train Loss: 0.6013 | Validation SMAPE: 59.71%
⏳ No improvement for 1 epoch(s)
Epoch 009/100 | Train Loss: 0.5978 | Validation SMAPE: 59.49%
⏳ No improvement for 2 epoch(s)
Ep

In [8]:
# ================================================================
# 1️⃣ Imports
# ================================================================
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np

# ================================================================
# 2️⃣ Train / Validation Split
# ================================================================
y = df["price"].values
y_log = np.log1p(y)

X_train, X_val, y_train, y_val = train_test_split(
    X_final, y_log, test_size=0.2, random_state=42
)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).to(device)

train_loader = DataLoader(
    TensorDataset(X_train_tensor, y_train_tensor),
    batch_size=64, shuffle=True
)
val_loader = DataLoader(
    TensorDataset(X_val_tensor, y_val_tensor),
    batch_size=64, shuffle=False
)

print(f"✅ Data prepared → Train: {X_train.shape}, Validation: {X_val.shape}")

# ================================================================
# 3️⃣ Define Residual Regressor
# ================================================================
class ResidualRegressor(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.block1 = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        self.block2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        self.block3 = nn.Sequential(
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        self.out = nn.Linear(128, 1)

        # Skip connections
        self.skip1 = nn.Linear(input_dim, 512)
        self.skip2 = nn.Linear(512, 256)

    def forward(self, x):
        x1 = self.block1(x) + self.skip1(x)
        x2 = self.block2(x1) + self.skip2(x1)
        x3 = self.block3(x2)
        return self.out(x3).squeeze(-1)

# ================================================================
# 4️⃣ Initialize Model
# ================================================================
model = ResidualRegressor(X_final.shape[1]).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)
criterion = nn.L1Loss()  # MAE in log-space

print(f"✅ Model initialized with {sum(p.numel() for p in model.parameters()):,} parameters")

# ================================================================
# 5️⃣ SMAPE Metric
# ================================================================
def smape(y_true, y_pred):
    y_true, y_pred = np.expm1(y_true), np.expm1(y_pred)
    return np.mean(np.abs(y_pred - y_true) / ((np.abs(y_pred) + np.abs(y_true)) / 2)) * 100

# ================================================================
# 6️⃣ Training Loop (100 Epochs + Early Stopping)
# ================================================================
EPOCHS = 100
patience = 5
best_smape = float("inf")
epochs_no_improve = 0
best_model_state = None

for epoch in range(EPOCHS):
    model.train()
    train_losses = []
    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

    # -----------------------
    # Validation Phase
    # -----------------------
    model.eval()
    val_preds, val_true = [], []
    with torch.no_grad():
        for xb, yb in val_loader:
            preds = model(xb)
            val_preds.extend(preds.cpu().numpy())
            val_true.extend(yb.cpu().numpy())

    val_preds, val_true = np.array(val_preds), np.array(val_true)
    smape_score = smape(val_true, val_preds)
    avg_train_loss = np.mean(train_losses)

    print(f"Epoch {epoch+1:03d}/{EPOCHS} | Train Loss: {avg_train_loss:.4f} | Validation SMAPE: {smape_score:.2f}%")

    # -----------------------
    # Early Stopping Logic
    # -----------------------
    if smape_score < best_smape - 0.1:
        best_smape = smape_score
        epochs_no_improve = 0
        best_model_state = model.state_dict()
        torch.save(model.state_dict(), "/kaggle/working/best_residual_regressor.pt")
        print(f"💾 New best model saved (SMAPE: {best_smape:.2f}%)")
    else:
        epochs_no_improve += 1
        print(f"⏳ No improvement for {epochs_no_improve} epoch(s)")

    if epochs_no_improve >= patience:
        print(f"🛑 Early stopping after {epoch+1} epochs. Best SMAPE: {best_smape:.2f}%")
        break

# ================================================================
# 7️⃣ Save Final Model
# ================================================================
if best_model_state:
    model.load_state_dict(best_model_state)
    torch.save(model.state_dict(), "/kaggle/working/best_residual_regressor.pt")

print("\n✅ Training complete — best model saved to /kaggle/working/best_residual_regressor.pt")


✅ Data prepared → Train: (60000, 785), Validation: (15000, 785)
✅ Model initialized with 1,102,337 parameters
Epoch 001/100 | Train Loss: 0.8560 | Validation SMAPE: 61.51%
💾 New best model saved (SMAPE: 61.51%)
Epoch 002/100 | Train Loss: 0.6473 | Validation SMAPE: 60.46%
💾 New best model saved (SMAPE: 60.46%)
Epoch 003/100 | Train Loss: 0.6228 | Validation SMAPE: 58.35%
💾 New best model saved (SMAPE: 58.35%)
Epoch 004/100 | Train Loss: 0.6103 | Validation SMAPE: 58.59%
⏳ No improvement for 1 epoch(s)
Epoch 005/100 | Train Loss: 0.5961 | Validation SMAPE: 57.65%
💾 New best model saved (SMAPE: 57.65%)
Epoch 006/100 | Train Loss: 0.5841 | Validation SMAPE: 56.77%
💾 New best model saved (SMAPE: 56.77%)
Epoch 007/100 | Train Loss: 0.5771 | Validation SMAPE: 56.87%
⏳ No improvement for 1 epoch(s)
Epoch 008/100 | Train Loss: 0.5691 | Validation SMAPE: 56.23%
💾 New best model saved (SMAPE: 56.23%)
Epoch 009/100 | Train Loss: 0.5611 | Validation SMAPE: 56.30%
⏳ No improvement for 1 epoch(s)
Ep

In [9]:
import torch
import torch.nn as nn

class TabTransformerRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim=512, n_heads=8, n_layers=2, dropout=0.3):
        super().__init__()
        self.input_fc = nn.Linear(input_dim, hidden_dim)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim,
            nhead=n_heads,
            dim_feedforward=hidden_dim * 2,
            dropout=dropout,
            activation="gelu",
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        self.mlp_head = nn.Sequential(
            nn.Linear(hidden_dim, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        # reshape for transformer: [batch, seq_len=1, hidden_dim]
        x = self.input_fc(x).unsqueeze(1)
        x = self.transformer(x)
        x = x.squeeze(1)
        return self.mlp_head(x).squeeze(-1)
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np

# Split and prepare tensors
y = df["price"].values
y_log = np.log1p(y)

X_train, X_val, y_train, y_val = train_test_split(
    X_final, y_log, test_size=0.2, random_state=42
)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).to(device)

train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=64, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=64, shuffle=False)

# Initialize model, optimizer, loss
model = TabTransformerRegressor(X_final.shape[1]).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1.5e-4, weight_decay=1e-5)
criterion = nn.L1Loss()

print(f"✅ Model initialized with {sum(p.numel() for p in model.parameters()):,} parameters")
def smape(y_true, y_pred):
    y_true, y_pred = np.expm1(y_true), np.expm1(y_pred)
    return np.mean(np.abs(y_pred - y_true) / ((np.abs(y_pred) + np.abs(y_true)) / 2)) * 100

EPOCHS = 100
patience = 5
best_smape = float("inf")
epochs_no_improve = 0
best_model_state = None

for epoch in range(EPOCHS):
    model.train()
    train_losses = []
    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

    # Validation
    model.eval()
    val_preds, val_true = [], []
    with torch.no_grad():
        for xb, yb in val_loader:
            preds = model(xb)
            val_preds.extend(preds.cpu().numpy())
            val_true.extend(yb.cpu().numpy())

    val_preds, val_true = np.array(val_preds), np.array(val_true)
    smape_score = smape(val_true, val_preds)
    avg_train_loss = np.mean(train_losses)

    print(f"Epoch {epoch+1:03d}/{EPOCHS} | Train Loss: {avg_train_loss:.4f} | Validation SMAPE: {smape_score:.2f}%")

    # Early stopping
    if smape_score < best_smape - 0.1:
        best_smape = smape_score
        epochs_no_improve = 0
        best_model_state = model.state_dict()
        torch.save(model.state_dict(), "/kaggle/working/best_tabtransformer_regressor.pt")
        print(f"💾 New best model saved (SMAPE: {best_smape:.2f}%)")
    else:
        epochs_no_improve += 1
        print(f"⏳ No improvement for {epochs_no_improve} epoch(s)")

    if epochs_no_improve >= patience:
        print(f"🛑 Early stopping after {epoch+1} epochs. Best SMAPE: {best_smape:.2f}%")
        break

if best_model_state:
    model.load_state_dict(best_model_state)
    torch.save(model.state_dict(), "/kaggle/working/best_tabtransformer_regressor.pt")

print("\n✅ Training complete — best model saved to /kaggle/working/best_tabtransformer_regressor.pt")


✅ Model initialized with 4,755,841 parameters
Epoch 001/100 | Train Loss: 0.7025 | Validation SMAPE: 64.60%
💾 New best model saved (SMAPE: 64.60%)
Epoch 002/100 | Train Loss: 0.6509 | Validation SMAPE: 64.23%
💾 New best model saved (SMAPE: 64.23%)
Epoch 003/100 | Train Loss: 0.6353 | Validation SMAPE: 61.28%
💾 New best model saved (SMAPE: 61.28%)
Epoch 004/100 | Train Loss: 0.6251 | Validation SMAPE: 63.45%
⏳ No improvement for 1 epoch(s)
Epoch 005/100 | Train Loss: 0.6177 | Validation SMAPE: 59.74%
💾 New best model saved (SMAPE: 59.74%)
Epoch 006/100 | Train Loss: 0.6119 | Validation SMAPE: 59.15%
💾 New best model saved (SMAPE: 59.15%)
Epoch 007/100 | Train Loss: 0.6052 | Validation SMAPE: 59.15%
⏳ No improvement for 1 epoch(s)
Epoch 008/100 | Train Loss: 0.5982 | Validation SMAPE: 62.11%
⏳ No improvement for 2 epoch(s)
Epoch 009/100 | Train Loss: 0.5967 | Validation SMAPE: 62.16%
⏳ No improvement for 3 epoch(s)
Epoch 010/100 | Train Loss: 0.5928 | Validation SMAPE: 59.79%
⏳ No improv

In [10]:

from copy import deepcopy
import torch.optim as optim

# Split data (log price)
y = np.log1p(df['price'].values)
X_train, X_val, y_train, y_val = train_test_split(X_final, y, test_size=0.2, random_state=42)

X_train_t = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_t = torch.tensor(y_train, dtype=torch.float32).to(device)
X_val_t = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val_t = torch.tensor(y_val, dtype=torch.float32).to(device)

train_loader = DataLoader(TensorDataset(X_train_t, y_train_t), batch_size=64, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_t, y_val_t), batch_size=64, shuffle=False)

# ============================================================
# Ensemble Configuration
# ============================================================
NUM_MODELS = 3
EPOCHS = 80
patience = 5
models = []
val_preds_all = []

for model_idx in range(NUM_MODELS):
    print(f"\n==============================")
    print(f"🚀 Training Model {model_idx+1}/{NUM_MODELS}")
    print(f"==============================")

    torch.manual_seed(42 + model_idx)
    np.random.seed(42 + model_idx)

    model = ResidualRegressor(X_final.shape[1]).to(device)
    optimizer = optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-4)
    criterion = nn.L1Loss()

    best_smape = float("inf")
    epochs_no_improve = 0
    best_state = None

    for epoch in range(EPOCHS):
        model.train()
        train_losses = []
        for xb, yb in train_loader:
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        model.eval()
        val_preds, val_true = [], []
        with torch.no_grad():
            for xb, yb in val_loader:
                preds = model(xb)
                val_preds.extend(preds.cpu().numpy())
                val_true.extend(yb.cpu().numpy())

        smape_score = smape(np.array(val_true), np.array(val_preds))
        avg_train_loss = np.mean(train_losses)
        print(f"Model {model_idx+1} | Epoch {epoch+1:03d}/{EPOCHS} | Train Loss: {avg_train_loss:.4f} | SMAPE: {smape_score:.2f}%")

        # Early stopping
        if smape_score < best_smape - 0.1:
            best_smape = smape_score
            epochs_no_improve = 0
            best_state = deepcopy(model.state_dict())
            torch.save(best_state, f"/kaggle/working/model_{model_idx+1}.pt")
            print(f"💾 Model {model_idx+1} improved (SMAPE: {best_smape:.2f}%)")
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print(f"🛑 Model {model_idx+1} early stopping at epoch {epoch+1}")
            break

    # Save and store best model
    model.load_state_dict(best_state)
    models.append(model)

    # Validation predictions for ensemble averaging
    model.eval()
    with torch.no_grad():
        preds = model(X_val_t).cpu().numpy()
    val_preds_all.append(preds)

# ============================================================
# Ensemble SMAPE Evaluation
# ============================================================
ensemble_preds = np.mean(val_preds_all, axis=0)
ensemble_smape = smape(y_val, ensemble_preds)
print(f"\n✅ Ensemble Validation SMAPE: {ensemble_smape:.2f}%")




🚀 Training Model 1/3
Model 1 | Epoch 001/80 | Train Loss: 0.8937 | SMAPE: 60.89%
💾 Model 1 improved (SMAPE: 60.89%)
Model 1 | Epoch 002/80 | Train Loss: 0.6475 | SMAPE: 59.27%
💾 Model 1 improved (SMAPE: 59.27%)
Model 1 | Epoch 003/80 | Train Loss: 0.6240 | SMAPE: 58.72%
💾 Model 1 improved (SMAPE: 58.72%)
Model 1 | Epoch 004/80 | Train Loss: 0.6083 | SMAPE: 57.96%
💾 Model 1 improved (SMAPE: 57.96%)
Model 1 | Epoch 005/80 | Train Loss: 0.5945 | SMAPE: 57.31%
💾 Model 1 improved (SMAPE: 57.31%)
Model 1 | Epoch 006/80 | Train Loss: 0.5859 | SMAPE: 58.05%
Model 1 | Epoch 007/80 | Train Loss: 0.5766 | SMAPE: 56.34%
💾 Model 1 improved (SMAPE: 56.34%)
Model 1 | Epoch 008/80 | Train Loss: 0.5683 | SMAPE: 56.55%
Model 1 | Epoch 009/80 | Train Loss: 0.5636 | SMAPE: 56.34%
Model 1 | Epoch 010/80 | Train Loss: 0.5558 | SMAPE: 56.74%
Model 1 | Epoch 011/80 | Train Loss: 0.5514 | SMAPE: 55.97%
💾 Model 1 improved (SMAPE: 55.97%)
Model 1 | Epoch 012/80 | Train Loss: 0.5455 | SMAPE: 55.53%
💾 Model 1 imp

In [11]:
# ================================================================
# ✅ FINAL INFERENCE PIPELINE — Residual Ensemble
# ================================================================
import pandas as pd, numpy as np, re, torch
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from scipy.sparse import hstack, csr_matrix
from tqdm.auto import tqdm
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)

# ================================================================
# 1️⃣ Load Test Data
# ================================================================
test_path = "/kaggle/input/amazon-2025-ml-dataset/test.csv"
test_df = pd.read_csv(test_path)
print(f"✅ Loaded test dataset: {test_df.shape[0]:,} rows × {test_df.shape[1]} columns")

# ================================================================
# 2️⃣ Feature Extraction from catalog_content
# ================================================================
def extract_features(text):
    if pd.isna(text):
        return pd.Series({"item_name": None, "bullet_points": None, "product_description": None, "value": None, "unit": None})
    item_names = re.findall(r"Item Name(?:\s*\d*)?:\s*(.*?)\s(?=Item Name|Bullet Point|Product Description:|Value:|Unit:|$)", text, re.S)
    item_name = item_names[-1].strip() if item_names else None
    bullet_points = re.findall(r"Bullet Point \d*:\s*(.*?)\s(?=Bullet Point \d+|Product Description:|Value:|Unit:|$)", text, re.S)
    bullet_points = " | ".join(bp.strip() for bp in bullet_points if bp.strip()) or None
    desc_match = re.search(r"Product Description:\s*(.*?)\s(?=Value:|Unit:|$)", text, re.S)
    product_description = desc_match.group(1).strip() if desc_match else None
    value_match = re.search(r"Value:\s*([0-9]*\.?[0-9]+)", text)
    value = float(value_match.group(1)) if value_match else None
    unit_match = re.search(r"Unit:\s*([A-Za-z\s\.]+)", text)
    unit = unit_match.group(1).strip() if unit_match else None
    return pd.Series({
        "item_name": item_name, "bullet_points": bullet_points,
        "product_description": product_description, "value": value, "unit": unit
    })

print("⏳ Extracting structured features...")
features = test_df["catalog_content"].apply(extract_features)
test_df = pd.concat([test_df, features], axis=1)
print("✅ Extraction complete!")

# ================================================================
# 3️⃣ Unit Normalization + Numeric Features
# ================================================================
unit_map = {
    'ounce':'ounce','ounces':'ounce','oz':'ounce','oz.':'ounce','Ounce':'ounce','OZ':'ounce',
    'pound':'pound','Pound':'pound','pounds':'pound','Pounds':'pound','lb':'pound','LB':'pound',
    'gram':'gram','grams':'gram','Gram':'gram','Grams':'gram','gramm':'gram','kg':'kilogram','K':'kilogram',
    'fl oz':'fl_ounce','Fl Oz':'fl_ounce','FL Oz':'fl_ounce','Fl.oz':'fl_ounce','Fluid Ounce':'fl_ounce',
    'Fluid Ounces':'fl_ounce','fluid ounce':'fl_ounce','fluid ounces':'fl_ounce',
    'milliliter':'milliliter','millilitre':'milliliter','mililitro':'milliliter',
    'ml':'milliliter','ltr':'liter','Liters':'liter','count':'count','Count':'count','COUNT':'count',
    'ct':'count','CT':'count','unit':'count','units':'count','Piece':'count','Each':'count','each':'count',
    'pack':'count','Pack':'count','PACK':'count','packs':'count','Packs':'count',
    'bottle':'container','Bottle':'container','bottles':'container','Bag':'container','bag':'container',
    'Box':'container','box':'container','Carton':'container','Jar':'container','Pouch':'container',
    'Bucket':'container','Can':'container','case':'container','CASE':'container'
}
test_df['unit_clean'] = test_df['unit'].map(unit_map).fillna('misc')

def map_unit_class(u):
    if u in ['ounce','fl_ounce','gram','milliliter','pound','kilogram','liter']:
        return 'quantity'
    elif u in ['count','container']:
        return 'count'
    else:
        return 'misc'

test_df['unit_class'] = test_df['unit_clean'].apply(map_unit_class)

unit_to_gram_equiv = {'ounce':28.35,'fl_ounce':29.57,'pound':453.59,
                      'gram':1.0,'milliliter':1.0,'kilogram':1000.0,'liter':1000.0}

test_df['std_value'] = test_df.apply(
    lambda x: x['value'] * unit_to_gram_equiv.get(x['unit_clean'], np.nan)
    if x['unit_class'] == 'quantity' else x['value'], axis=1)
test_df['log_value'] = np.log1p(test_df['value'])
test_df['log_std_value'] = np.log1p(test_df['std_value'])

# ================================================================
# 4️⃣ DistilBERT Embeddings
# ================================================================
test_df['text_all'] = (
    test_df['item_name'].fillna('') + ' ' +
    test_df['bullet_points'].fillna('') + ' ' +
    test_df['product_description'].fillna('')
)

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
distilbert_model = DistilBertModel.from_pretrained("distilbert-base-uncased").to(device)
distilbert_model.eval()

def get_distilbert_embedding(text, max_len=64):
    tokens = tokenizer(text, return_tensors="pt", truncation=True,
                       padding="max_length", max_length=max_len)
    tokens = {k: v.to(device) for k, v in tokens.items()}
    with torch.no_grad():
        outputs = distilbert_model(**tokens)
    emb = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
    return emb

print("⏳ Encoding text with DistilBERT ...")
texts = test_df['text_all'].tolist()
distilbert_embeddings = np.vstack([get_distilbert_embedding(t) for t in tqdm(texts, desc="BERT Encoding")])
print("✅ DistilBERT embeddings shape:", distilbert_embeddings.shape)

# ================================================================
# 5️⃣ Combine All Features
# ================================================================
from sklearn.preprocessing import StandardScaler, OneHotEncoder

num_cols = ['value', 'log_value', 'std_value', 'log_std_value']
scaler = StandardScaler()
X_num = scaler.fit_transform(test_df[num_cols].fillna(0))

cat_cols = ['unit_clean', 'unit_class']
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_cat = ohe.fit_transform(test_df[cat_cols])

X_final_test = np.hstack([distilbert_embeddings, X_num, X_cat])
print("✅ Final Test Feature Matrix:", X_final_test.shape)

# ================================================================
# 6️⃣ Define Model Architecture (same as training)
# ================================================================
class ResidualRegressor(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.block1 = nn.Sequential(
            nn.Linear(input_dim, 512), nn.BatchNorm1d(512), nn.ReLU(), nn.Dropout(0.3)
        )
        self.block2 = nn.Sequential(
            nn.Linear(512, 256), nn.BatchNorm1d(256), nn.ReLU(), nn.Dropout(0.3)
        )
        self.block3 = nn.Sequential(
            nn.Linear(256, 128), nn.BatchNorm1d(128), nn.ReLU(), nn.Dropout(0.2)
        )
        self.out = nn.Linear(128, 1)
        self.skip1 = nn.Linear(input_dim, 512)
        self.skip2 = nn.Linear(512, 256)
    def forward(self, x):
        x1 = self.block1(x) + self.skip1(x)
        x2 = self.block2(x1) + self.skip2(x1)
        x3 = self.block3(x2)
        return self.out(x3).squeeze(-1)

# ================================================================
# 7️⃣ Load Ensemble Models & Predict
# ================================================================
NUM_MODELS = 3
preds_all = []

X_tensor = torch.tensor(X_final_test, dtype=torch.float32).to(device)

for i in range(NUM_MODELS):
    model_path = f"/kaggle/working/model_{i+1}.pt"
    model = ResidualRegressor(X_final_test.shape[1]).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    with torch.no_grad():
        preds_log = model(X_tensor).cpu().numpy()
        preds_all.append(np.expm1(preds_log))  # revert log1p()

# ================================================================
# 8️⃣ Ensemble Average & Postprocess
# ================================================================
final_preds = np.mean(preds_all, axis=0)
final_preds = np.clip(final_preds, a_min=0.1, a_max=None)

# ================================================================
# 9️⃣ Save Submission
# ================================================================
submission = pd.DataFrame({
    "sample_id": test_df["sample_id"],
    "price": final_preds
})
output_path = "/kaggle/working/test_out.csv"
submission.to_csv(output_path, index=False)

print("\n✅ Ensemble inference complete!")
print(f"✅ Saved submission → {output_path}")
print(submission.head())


✅ Using device: cuda
✅ Loaded test dataset: 75,000 rows × 3 columns
⏳ Extracting structured features...
✅ Extraction complete!


/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


⏳ Encoding text with DistilBERT ...


BERT Encoding:   0%|          | 0/75000 [00:00<?, ?it/s]

✅ DistilBERT embeddings shape: (75000, 768)
✅ Final Test Feature Matrix: (75000, 785)


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(



✅ Ensemble inference complete!
✅ Saved submission → /kaggle/working/test_out.csv
   sample_id      price
0     100179  16.199829
1     245611  15.210381
2     146263  23.122168
3      95658  41.876678
4      36806  27.598959
